In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.externals import joblib
import sklearn

In [2]:
# Creating DF with features and duplicate variable only for final model
def append_df(df1, df2):
    # df1 with column names, and df2 without
    df2.columns = df1.columns
    df1 = df1.append(df2, ignore_index=True)
    return df1

In [39]:
test_data_cols1 = pd.read_csv('test_data_cols1.csv', sep=',')
test_data_cols2 = pd.read_csv('test_data_cols2.csv', sep=',', skiprows=1, header=None)
test_data_cols1 = append_df(test_data_cols1,test_data_cols2) 
test_data_cols1.count()

test_id            2345796
q1_form_len        2345796
q2_form_len        2345796
q1_length          2345796
q2_length          2345796
q1_unique          2345796
q2_unique          2345796
q1_form_uni        2345796
q2_form_uni        2345796
q1_form_char       2345796
q2_form_char       2345796
q1_q2_char_diff    2345796
common_cnt         2345796
prcnt_common       2345379
dtype: int64

In [40]:
test_data_cols1['prcnt_common'].fillna(test_data_cols1['prcnt_common'].mean(), inplace=True)

In [41]:
test_data_cols1.count()

test_id            2345796
q1_form_len        2345796
q2_form_len        2345796
q1_length          2345796
q2_length          2345796
q1_unique          2345796
q2_unique          2345796
q1_form_uni        2345796
q2_form_uni        2345796
q1_form_char       2345796
q2_form_char       2345796
q1_q2_char_diff    2345796
common_cnt         2345796
prcnt_common       2345796
dtype: int64

## Test Score

In [43]:
def test_clean(test_data):
    # Create Test_ID DF
    test_id_col = test_data["test_id"]
    test_id_col = pd.DataFrame(pd.Series(test_id_col))
    
    # Clean Test DF model
    test_data.drop(['test_id'], axis=1, inplace=True)
    #test_data.drop(['prcnt_common'], axis=1, inplace=True)               # Include this column in the main version-Comment it
    
    return test_id_col, test_data

In [44]:
# Run this only for the basic only features case
test_id_col, test_data_cols1 = test_clean(test_data_cols1)

In [ ]:
# Below function for:
# 1. Take the Model Path and Name : Load the Model
# 2. Take the Test Data with iteration of desired featuers
# 3. Take the Test_ID column dataframe
# 4. Take the final submission path and file name to be submitted on Kaggle

In [20]:

def test_df_create(model_path, model_name, test_data, test_id_col, submission_path, final_file_name):
    # Loading the Desired Model
    xgb2 = joblib.load(model_path+model_name)
    
    # Run model on Test Data
    test_pred_xgb2 = xgb2.predict(test_data)
    test_pred_xgb2 = pd.DataFrame(test_pred_xgb2)
    test_pred_xgb2.columns=['is_duplicate']
    
    # Final Submission DF
    submit_df = pd.concat([test_id_col,test_pred_xgb2 ], axis=1)
    submit_df.to_csv(test_submission_path+final_file_name, index=False, sep=',')
    
    # Basis info
    print "Test DF Shape",(submit_df.shape[0], submit_df.shape[1])
    print(submit_df.head())
    print('Total 1s : %d' % (test_pred_xgb2.sum()))
    print('1s Percentage : %f' % (float(test_pred_xgb2.sum())/float(submit_df.shape[0])*100))
    
    return(submit_df)

In [21]:
from sklearn.externals import joblib
def test_df_create_prob(model_path, model_name, test_data, test_id_col, submission_path, final_file_name):
    # Loading the Desired Model
    xgb2 = joblib.load(model_path+model_name)
    
    # Run model on Test Data
    test_pred_xgb2 = xgb2.predict_proba(test_data)
    test_pred_xgb2 = pd.DataFrame(test_pred_xgb2)
    test_pred_xgb2 = pd.DataFrame(test_pred_xgb2[1])
    test_pred_xgb2.columns=['is_duplicate']
    
    # Final Submission DF
    submit_df = pd.concat([test_id_col,test_pred_xgb2 ], axis=1)
    submit_df.to_csv(test_submission_path+final_file_name, index=False, sep=',')
    
    # Basis info
    print "Test DF Shape",(submit_df.shape[0], submit_df.shape[1])
    print(submit_df.head())
    print('Total 1s : %d' % (test_pred_xgb2.sum()))
    print('1s Percentage : %f' % (float(test_pred_xgb2.sum())/float(submit_df.shape[0])*100))
    
    return(submit_df)

## Scoring on Model of Basic Features (roc_auc)

In [42]:
model_path='/home/pgoyal/quora/model/'
test_submission_path= '/home/pgoyal/quora/'

In [31]:
# Model-1 :
# 1. Basic features only
# 2. Hyperparameters set using a validation split data - roc_auc
# 3. Final model is trained with the step-2 parameters and trained on the training dataset only (validation part not included!)

In [112]:
# Model-1 : Choose this one only as of now (Score 8.69~) : Model without prcnt_common feature
model_name= 'xgb2_dup_auc.pkl'
file_name = 'test_submission_1.csv'
final_df = test_df_create(model_path, model_name, test_data_cols1, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0             0
1        1             1
2        2             1
3        3             0
4        4             0
Total 1s : 356786
1s Percentage : 15.209592


In [32]:
test_data_cols1.drop(['prcnt_common'], inplace=True, axis=1)

In [33]:
model_path='/home/pgoyal/quora/model/'
test_submission_path= '/home/pgoyal/quora/'

# Model-1b : With Probability Score, Not chosen (Score 0.46741~) : Model without prcnt_common feature
model_name= 'xgb2_dup_auc.pkl'
file_name = 'test_submission_1b_7.csv'
final_df = test_df_create_prob(model_path, model_name, test_data_cols1, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0      0.202753
1        1      0.518799
2        2      0.668572
3        3      0.151815
4        4      0.179836
Total 1s : 596783
1s Percentage : 25.440573


In [ ]:
# Model-2
# 1. Basic features & prcnt_common feature also  (So, load the test_data_cols1 dataset again and run this, as prcnt_common is dropped)
# 2. Hyperparameters set using a validation split data - roc_auc
# 3. Final model is trained with the step-2 parameters and trained on the training dataset only (validation part not included!)

In [123]:
# Model-2 : Not Chosen (Score 14.~)  : Model with prcnt_common
model_name= 'xgb2_dup_auc_2.pkl'
file_name = 'test_submission_2.csv'
final_df = test_df_create(model_path, model_name, test_data_cols1, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0             0
1        1             1
2        2             1
3        3             0
4        4             0
Total 1s : 928668
1s Percentage : 39.588609


In [46]:
# Model-2 : (Model 2 only with probability scores): Not chosen 0.61042 : Model with prcnt_common
model_name= 'xgb2_dup_auc_2.pkl'
file_name = 'test_submission_2b_8.csv'
final_df = test_df_create_prob(model_path, model_name, test_data_cols1, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0      0.358872
1        1      0.654270
2        2      0.783266
3        3      0.260690
4        4      0.307835
Total 1s : 837069
1s Percentage : 35.683831


In [151]:
#del test_data_cols1

## Scoring on Model of Basic+Fuzzy Features (roc_auc)

In [3]:
# Loading the test data with the desired basic and Fuzzy features
test_basic_fuzzy1 = pd.read_csv('/home/pgoyal/quora/test_basic_fuzzy1.csv', sep=',')
for i in [2,3,4,5,6]:
    vars()['test_basic_fuzzy'+str(i)] = pd.read_csv('/home/pgoyal/quora/test_basic_fuzzy'+str(i)+'.csv',  sep=',', skiprows=1, header=None)

In [4]:
test_basic_fuzzy1 = append_df(test_basic_fuzzy1, test_basic_fuzzy2)
test_basic_fuzzy1 = append_df(test_basic_fuzzy1, test_basic_fuzzy3)
test_basic_fuzzy1 = append_df(test_basic_fuzzy1, test_basic_fuzzy4)
test_basic_fuzzy1 = append_df(test_basic_fuzzy1, test_basic_fuzzy5)
test_basic_fuzzy1 = append_df(test_basic_fuzzy1, test_basic_fuzzy6)

In [5]:
test_basic_fuzzy1.head()

,test_id,question1_form,question2_form,q1_form_len,q2_form_len,q1_length,q2_length,q1_unique,q2_unique,q1_form_uni,...,q1_q2_char_diff,common_cnt,prcnt_common,partial_ratio,partial_token_set_ratio,partial_token_sort_ratio,q_ratio,token_set_ratio,token_sort_ratio,w_ratio
0,0,surfac pro compar ipad pro,microsoft choos core core home surfac pro,5,7,11,14,11,13,4,...,13,2,0.20,56,100,67,30,63,54,86
1,1,hair transplant age much cost,much cost hair transplant requir,5,5,14,7,14,7,5,...,3,4,0.40,71,100,76,59,93,82,88
2,2,best way send money china us,send money china,6,3,14,6,12,6,6,...,9,3,0.33,100,100,100,73,100,73,90
3,3,food emulsifi,food fibr,2,2,4,3,4,3,2,...,4,1,0.25,56,100,88,64,64,64,64
4,4,aberystwyth start read,start read,3,2,4,6,4,6,3,...,11,2,0.40,100,100,100,63,100,63,90


In [34]:
model_path='/home/pgoyal/quora/model2/'
test_submission_path= '/home/pgoyal/quora/'

In [11]:
print test_basic_fuzzy1.columns
print test_basic_fuzzy1.shape

Index([u'test_id', u'question1_form', u'question2_form', u'q1_form_len',
       u'q2_form_len', u'q1_length', u'q2_length', u'q1_unique', u'q2_unique',
       u'q1_form_uni', u'q2_form_uni', u'q1_form_char', u'q2_form_char',
       u'q1_q2_char_diff', u'common_cnt', u'prcnt_common', u'partial_ratio',
       u'partial_token_set_ratio', u'partial_token_sort_ratio', u'q_ratio',
       u'token_set_ratio', u'token_sort_ratio', u'w_ratio'],
      dtype='object')
(2345796, 23)


In [12]:
test_id_col = test_basic_fuzzy1["test_id"]
test_id_col = pd.DataFrame(pd.Series(test_id_col))

In [13]:
test_basic_fuzzy1.drop(['question1_form','question2_form','test_id'],inplace=True, axis=1)

In [ ]:
# Model-3 : 
# 1. Basic and Fuzzy features
# 2. Hyperparameters set using a validation split data - roc_auc
# 3. Final model is trained with the step-2 parameters and trained on the training dataset only (validation part not included!)

In [202]:
# Model-3 : Model with Basic and Fuzzy features with validation split : Not chosen (8.96)
model_name= 'xgb2_dup_auc_2.pkl'
file_name = 'test_submission_3.csv'
final_df = test_df_create(model_path, model_name, test_basic_fuzzy1, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0             0
1        1             1
2        2             1
3        3             0
4        4             0
Total 1s : 441040
1s Percentage : 18.801294


In [16]:
# Model-3b : (Model 3 only with probability scores): Not chosen 0.45475
model_name= 'xgb2_dup_auc_2.pkl'
file_name = 'test_submission_3b_6.csv'
final_df = test_df_create_prob(model_path, model_name, test_basic_fuzzy1, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0      0.120734
1        1      0.588195
2        2      0.684597
3        3      0.279819
4        4      0.193473
Total 1s : 587186
1s Percentage : 25.031442


In [29]:
# Model-4 : 
# 1. Basic and Fuzzy features
# 2. Hyperparameters set using a validation split data - roc_auc
# 3. Final model is trained with the step-2 parameters and trained on the overall data

In [204]:
# Model-4 : Not at top
model_name= 'xgb_dup_auc_final.pkl'
file_name = 'test_submission_4.csv'
final_df = test_df_create(model_path, model_name, test_basic_fuzzy1, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0             0
1        1             1
2        2             1
3        3             0
4        4             0
Total 1s : 438643
1s Percentage : 18.699111


In [35]:
# Model-4b :  (Model 4 only with probability scores) : Chosen 0.45414
model_name= 'xgb_dup_auc_final.pkl'
file_name = 'test_submission_4b_5.csv'
final_df = test_df_create_prob(model_path, model_name, test_basic_fuzzy1, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0      0.112291
1        1      0.610998
2        2      0.679616
3        3      0.292897
4        4      0.206938
Total 1s : 586805
1s Percentage : 25.015189


## Scoring on Model of Basic+Fuzzy Features (log_loss)

In [54]:
# Loading the test data with the desired basic and Fuzzy features (Issue of file upload size, so broken in chunks and loaded)
test_basic_fuzzy1 = pd.read_csv('/home/pgoyal/quora/test_basic_fuzzy1.csv', sep=',')
for i in [2,3,4,5,6]:
    vars()['test_basic_fuzzy'+str(i)] = pd.read_csv('/home/pgoyal/quora/test_basic_fuzzy'+str(i)+'.csv',  sep=',', skiprows=1, header=None)

In [55]:
test_basic_fuzzy1 = append_df(test_basic_fuzzy1, test_basic_fuzzy2)
test_basic_fuzzy1 = append_df(test_basic_fuzzy1, test_basic_fuzzy3)
test_basic_fuzzy1 = append_df(test_basic_fuzzy1, test_basic_fuzzy4)
test_basic_fuzzy1 = append_df(test_basic_fuzzy1, test_basic_fuzzy5)
test_basic_fuzzy1 = append_df(test_basic_fuzzy1, test_basic_fuzzy6)
print test_basic_fuzzy1.shape
test_basic_fuzzy1.head()

(2345796, 23)


,test_id,question1_form,question2_form,q1_form_len,q2_form_len,q1_length,q2_length,q1_unique,q2_unique,q1_form_uni,...,q1_q2_char_diff,common_cnt,prcnt_common,partial_ratio,partial_token_set_ratio,partial_token_sort_ratio,q_ratio,token_set_ratio,token_sort_ratio,w_ratio
0,0,surfac pro compar ipad pro,microsoft choos core core home surfac pro,5,7,11,14,11,13,4,...,13,2,0.20,56,100,67,30,63,54,86
1,1,hair transplant age much cost,much cost hair transplant requir,5,5,14,7,14,7,5,...,3,4,0.40,71,100,76,59,93,82,88
2,2,best way send money china us,send money china,6,3,14,6,12,6,6,...,9,3,0.33,100,100,100,73,100,73,90
3,3,food emulsifi,food fibr,2,2,4,3,4,3,2,...,4,1,0.25,56,100,88,64,64,64,64
4,4,aberystwyth start read,start read,3,2,4,6,4,6,3,...,11,2,0.40,100,100,100,63,100,63,90


In [6]:
model_path='/home/pgoyal/quora/model3/'
test_submission_path= '/home/pgoyal/quora/'
test_id_col = test_basic_fuzzy1["test_id"]
test_id_col = pd.DataFrame(pd.Series(test_id_col))
test_basic_fuzzy1.drop(['question1_form','question2_form','test_id'],inplace=True, axis=1)

In [ ]:
# Model-5 : 
# 1. Basic and Fuzzy features
# 2. Hyperparameters set using a validation split data - log_loss (initial hyperparam from Model-4)
# 3. Final model is trained with the step-2 parameters and trained on the training dataset only 

In [58]:
# Model-5 : (validation part not included!)
model_name= 'xgb_log2_dup_logloss.pkl'
file_name = 'test_submission_5.csv'
final_df = test_df_create_prob(model_path, model_name, test_basic_fuzzy1, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0      0.117726
1        1      0.604464
2        2      0.706178
3        3      0.258797
4        4      0.195440
Total 1s : 583877
1s Percentage : 24.890365


In [59]:
# Model-5b : (with prediction score in the final output and trained on full dataset)
model_name= 'xgb_log3_dup_logloss.pkl'
file_name = 'test_submission_5b_9.csv'
final_df = test_df_create_prob(model_path, model_name, test_basic_fuzzy1, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0      0.123649
1        1      0.625843
2        2      0.684863
3        3      0.288893
4        4      0.218868
Total 1s : 584091
1s Percentage : 24.899480


In [8]:
# Model-5c : (with 0 or 1 in the final output)
model_name= 'xgb_log3_dup_logloss.pkl'
file_name = 'test_submission_5c_10.csv'
final_df = test_df_create(model_path, model_name, test_basic_fuzzy1, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0             0
1        1             1
2        2             1
3        3             0
4        4             0
Total 1s : 441179
1s Percentage : 18.807219


## Scoring on Model of Basic+Fuzzy+Similarity Features (log_loss)

In [3]:
# Loading the test data with the desired basic and Fuzzy features (Issue of file upload size, so broken in chunks and loaded)
test_basic_fuzzy1 = pd.read_csv('/home/pgoyal/quora/test_basic_fuzzy1.csv', sep=',')
for i in [2,3,4,5,6]:
    vars()['test_basic_fuzzy'+str(i)] = pd.read_csv('/home/pgoyal/quora/test_basic_fuzzy'+str(i)+'.csv',  sep=',', skiprows=1, header=None)

test_basic_fuzzy1 = append_df(test_basic_fuzzy1, test_basic_fuzzy2)
test_basic_fuzzy1 = append_df(test_basic_fuzzy1, test_basic_fuzzy3)
test_basic_fuzzy1 = append_df(test_basic_fuzzy1, test_basic_fuzzy4)
test_basic_fuzzy1 = append_df(test_basic_fuzzy1, test_basic_fuzzy5)
test_basic_fuzzy1 = append_df(test_basic_fuzzy1, test_basic_fuzzy6)
print test_basic_fuzzy1.shape
test_basic_fuzzy1.head()

(2345796, 23)


,test_id,question1_form,question2_form,q1_form_len,q2_form_len,q1_length,q2_length,q1_unique,q2_unique,q1_form_uni,...,q1_q2_char_diff,common_cnt,prcnt_common,partial_ratio,partial_token_set_ratio,partial_token_sort_ratio,q_ratio,token_set_ratio,token_sort_ratio,w_ratio
0,0,surfac pro compar ipad pro,microsoft choos core core home surfac pro,5,7,11,14,11,13,4,...,13,2,0.20,56,100,67,30,63,54,86
1,1,hair transplant age much cost,much cost hair transplant requir,5,5,14,7,14,7,5,...,3,4,0.40,71,100,76,59,93,82,88
2,2,best way send money china us,send money china,6,3,14,6,12,6,6,...,9,3,0.33,100,100,100,73,100,73,90
3,3,food emulsifi,food fibr,2,2,4,3,4,3,2,...,4,1,0.25,56,100,88,64,64,64,64
4,4,aberystwyth start read,start read,3,2,4,6,4,6,3,...,11,2,0.40,100,100,100,63,100,63,90


In [6]:
# Load and merge another test dataset
test_w2v_feats1 = pd.read_csv('/home/pgoyal/quora/test_w2v_feats1.csv', sep=',')
for i in [2,3]:
    vars()['test_w2v_feats'+str(i)] = pd.read_csv('/home/pgoyal/quora/test_w2v_feats'+str(i)+'.csv',  sep=',', skiprows=1, header=None)

test_w2v_feats1 = append_df(test_w2v_feats1, test_w2v_feats2)
test_w2v_feats1 = append_df(test_w2v_feats1, test_w2v_feats3)
print test_w2v_feats1.shape
test_w2v_feats1.head()

(2345796, 16)


,test_id,q1_freq_in_q1,q2_freq_in_q2,q1_freq_in_q1q2,q2_freq_in_q1q2,cosine_val,cityblock_val,jaccard_val,canberra_val,euclidean_val,minkowski_val,braycurtis_val,skew_que1,skew_que2,kurt_que1,kurt_que2
0,0,1,1,1,1,0.94,45.27,1.0,91.10,3.32,1.51,0.17,0.15,0.04,-0.19,-0.07
1,1,1,1,2,2,0.98,27.29,1.0,72.43,1.93,0.86,0.11,0.04,0.07,-0.01,0.13
2,2,1,1,1,1,0.96,36.57,1.0,87.04,2.68,1.22,0.14,0.03,-0.01,-0.04,-0.13
3,3,1,1,1,1,0.90,65.77,1.0,112.34,4.67,2.09,0.23,0.05,0.10,0.14,-0.10
4,4,1,1,1,1,0.95,42.86,1.0,91.51,3.04,1.36,0.17,0.07,0.15,-0.06,-0.08


In [7]:
print test_basic_fuzzy1.columns
print test_basic_fuzzy1.shape
print test_w2v_feats1.columns
print test_w2v_feats1.shape

Index([u'test_id', u'question1_form', u'question2_form', u'q1_form_len',
       u'q2_form_len', u'q1_length', u'q2_length', u'q1_unique', u'q2_unique',
       u'q1_form_uni', u'q2_form_uni', u'q1_form_char', u'q2_form_char',
       u'q1_q2_char_diff', u'common_cnt', u'prcnt_common', u'partial_ratio',
       u'partial_token_set_ratio', u'partial_token_sort_ratio', u'q_ratio',
       u'token_set_ratio', u'token_sort_ratio', u'w_ratio'],
      dtype='object')
(2345796, 23)
Index([u'test_id', u'q1_freq_in_q1', u'q2_freq_in_q2', u'q1_freq_in_q1q2',
       u'q2_freq_in_q1q2', u'cosine_val', u'cityblock_val', u'jaccard_val',
       u'canberra_val', u'euclidean_val', u'minkowski_val', u'braycurtis_val',
       u'skew_que1', u'skew_que2', u'kurt_que1', u'kurt_que2'],
      dtype='object')
(2345796, 16)


In [8]:
test_basic_fuzzy1.drop(['question1_form','question2_form'], axis=1, inplace=True)

In [9]:
# Join both the dataframes test_basic_fuzzy1 and test_w2v_feats1 on 'test_id' column
test_final = pd.merge(test_w2v_feats1, test_basic_fuzzy1, left_on=['test_id'],right_on=['test_id'],how='inner', right_index=False)
print test_final.columns

Index([u'test_id', u'q1_freq_in_q1', u'q2_freq_in_q2', u'q1_freq_in_q1q2',
       u'q2_freq_in_q1q2', u'cosine_val', u'cityblock_val', u'jaccard_val',
       u'canberra_val', u'euclidean_val', u'minkowski_val', u'braycurtis_val',
       u'skew_que1', u'skew_que2', u'kurt_que1', u'kurt_que2', u'q1_form_len',
       u'q2_form_len', u'q1_length', u'q2_length', u'q1_unique', u'q2_unique',
       u'q1_form_uni', u'q2_form_uni', u'q1_form_char', u'q2_form_char',
       u'q1_q2_char_diff', u'common_cnt', u'prcnt_common', u'partial_ratio',
       u'partial_token_set_ratio', u'partial_token_sort_ratio', u'q_ratio',
       u'token_set_ratio', u'token_sort_ratio', u'w_ratio'],
      dtype='object')


In [12]:
model_path='/home/pgoyal/quora/model4/'
test_submission_path= '/home/pgoyal/quora/'
test_id_col = test_final["test_id"]
test_id_col = pd.DataFrame(pd.Series(test_id_col))
test_final.drop(['test_id'],inplace=True, axis=1)

In [13]:
# Check the final columns order here
print test_final.shape
print test_final.columns

(2345796, 35)
Index([u'q1_freq_in_q1', u'q2_freq_in_q2', u'q1_freq_in_q1q2',
       u'q2_freq_in_q1q2', u'cosine_val', u'cityblock_val', u'jaccard_val',
       u'canberra_val', u'euclidean_val', u'minkowski_val', u'braycurtis_val',
       u'skew_que1', u'skew_que2', u'kurt_que1', u'kurt_que2', u'q1_form_len',
       u'q2_form_len', u'q1_length', u'q2_length', u'q1_unique', u'q2_unique',
       u'q1_form_uni', u'q2_form_uni', u'q1_form_char', u'q2_form_char',
       u'q1_q2_char_diff', u'common_cnt', u'prcnt_common', u'partial_ratio',
       u'partial_token_set_ratio', u'partial_token_sort_ratio', u'q_ratio',
       u'token_set_ratio', u'token_sort_ratio', u'w_ratio'],
      dtype='object')


In [ ]:
# Model-6 : 
# 1. Basic, Fuzzy and Similarity features
# 2. Hyperparameters set using a validation split data - log_loss (initial hyperparam from Model-4)
# 3. Final model is trained with the step-2 parameters and trained on the training dataset only (validation part not included!)

In [18]:
# Model-6 : Model with GridSearch till param8 only (validation part not included!)
model_name= 'xgb_log2_dup_logloss.pkl'
file_name = 'test_submission_6.csv'
final_df = test_df_create_prob(model_path, model_name, test_final, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0      0.025245
1        1      0.517904
2        2      0.467295
3        3      0.152954
4        4      0.117440
Total 1s : 339378
1s Percentage : 14.467504


In [19]:
# Model-6 : Model with GridSearch till param8 only (validation part included!)
model_name= 'xgb_dup_logloss.pkl'
file_name = 'test_submission_6f_16.csv'
final_df = test_df_create_prob(model_path, model_name, test_final, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0      0.025391
1        1      0.573217
2        2      0.497429
3        3      0.237450
4        4      0.104594
Total 1s : 340001
1s Percentage : 14.494058


In [22]:
# Model-6 : Model with GridSearch till param9 with best set (validation part not included!)
model_name= 'xgb_log2b_dup_logloss.pkl'
file_name = 'test_submission_6c_13.csv'
final_df = test_df_create_prob(model_path, model_name, test_final, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0      0.027858
1        1      0.477014
2        2      0.527143
3        3      0.205686
4        4      0.120535
Total 1s : 334116
1s Percentage : 14.243189


In [23]:
# Model-6b : (with prediction score in the final output and trained on full dataset with param9)
model_name= 'xgb_log3_dup_logloss.pkl'
file_name = 'test_submission_6b_12.csv'
final_df = test_df_create_prob(model_path, model_name, test_final, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0      0.017497
1        1      0.550115
2        2      0.470525
3        3      0.209268
4        4      0.109565
Total 1s : 334537
1s Percentage : 14.261169


In [24]:
# Model-6c : (with prediction binary in the final output and trained on full dataset with param9)
model_name= 'xgb_log3_dup_logloss.pkl'
file_name = 'test_submission_6e_15.csv'
final_df = test_df_create(model_path, model_name, test_final, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0             0
1        1             1
2        2             0
3        3             0
4        4             0
Total 1s : 151082
1s Percentage : 6.440543


In [17]:
# Model-6d :  Hyperparameter tuning NOT DONE (with prediction score in the final output and trained on train part of dataset)
model_name= 'xgb_log1_dup_logloss.pkl'
file_name = 'test_submission_6d_14.csv'
final_df = test_df_create_prob(model_path, model_name, test_final, test_id_col,  test_submission_path, file_name)

Test DF Shape (2345796, 2)
   test_id  is_duplicate
0        0      0.025254
1        1      0.588287
2        2      0.488603
3        3      0.195160
4        4      0.123961
Total 1s : 341539
1s Percentage : 14.559635


In [ ]:
# Run the Geometric mean ensemble script on the following solutions as these are the best ones till now  :
# test_submission_6.csv
# test_submission_6b_12.csv
# test_submission_6c_13.csv
# test_submission_6d_14.csv
# test_submission_6f_16.csv

# Final output file : test_submission_geomean.csv

# Tried with Rank Average script as well : but not better results as compared to geom mean

In [ ]:
# Try with weight ensemble techniques later on 